In [4]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [5]:
import nltk
nltk.download('book')


[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown.zip.
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\chat80.zip.
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\conll2000.zip.
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping c

True

In [1]:
import math
import random
from collections import Counter, defaultdict
import nltk
import re # для обробки токенів при генерації

nltk.download('brown')
nltk.download('punkt')
nltk.download('gutenberg')

#Спеціальні токени
START_TOKEN = '<s>' # Початок речення
END_TOKEN = '</s>' # Кінець речення
UNK_TOKEN = '<unk>' # Невідоме слово

def load_nltk_sentences(corpus_name, file_ids=None):
    print(f"Завантаження корпусу '{corpus_name}'...")
    try:
        if corpus_name == 'brown':
            sents = nltk.corpus.brown.sents()
        elif corpus_name == 'gutenberg':
            sents = nltk.corpus.gutenberg.sents(file_ids)
        else:
            raise ValueError(f"Непідтримуваний корпус: {corpus_name}")
    except nltk.downloader.DownloadError:
         print(f"Помилка завантаження корпусу '{corpus_name}'.")
         return [] # повертаємо порожній список, якщо завантаження не вдалося

    # переведення токенів у нижній регістр
    sentences = [[token.lower() for token in sent] for sent in sents]
    print(f"Завантажено {len(sentences)} речень.")
    return sentences

# Базовий клас N-грамної моделі (з UNK)
class BaseNGramModel:
    def __init__(self, n: int, unk_threshold: int = 1):
        """
        n: максимальний рівень N-грамів.
        unk_threshold: поріг частоти для UNK.
        """
        if n < 1: raise ValueError("N >= 1")
        self.n = n
        self.unk_threshold = unk_threshold

        self._ngram_counts = {} # Лічильники {рівень: Counter}
        self._word_counts_raw = Counter() # Сирі частоти слів (для визначення UNK)
        self.vocabulary = set() # Словник моделі (включаючи спецтокени, UNK)
        self._total_tokens = 0 # Загальна кількість токенів після UNK/спецтокенів

    def fit(self, sentences: list[list[str]]):
        print(f"Навчання моделі (N={self.n}, UNK Threshold={self.unk_threshold})...")

        #Підрахунок сирих частот слів для визначення UNK
        for sent in sentences: self._word_counts_raw.update(sent)
        unk_words = {word for word, freq in self._word_counts_raw.items() if freq <= self.unk_threshold}

        # Формування словника моделі
        self.vocabulary.update({START_TOKEN, END_TOKEN, UNK_TOKEN})
        self.vocabulary.update(word for word in self._word_counts_raw if word not in unk_words)
        print(f"  Розмір словника (включаючи спецтокени, UNK): {len(self.vocabulary)}")

        #Обробка речень: додавання START/END, заміна UNK
        processed_sentences = []
        unigram_counts = Counter() # Підрахунок уніграм після обробки
        for sent in sentences:
            processed_tokens = [self._map_token_to_vocab(token) for token in sent] # Мапінг на UNK
            sentence_with_specials = [START_TOKEN] * (self.n - 1) + processed_tokens + [END_TOKEN]
            processed_sentences.append(sentence_with_specials)
            unigram_counts.update(sentence_with_specials)

        # Зберігаємо уніграми (рівень 1) та загальну кількість токенів
        self._ngram_counts[1] = unigram_counts
        self._total_tokens = sum(self._ngram_counts[1].values())
        print(f"  Загальна кількість токенів після обробки: {self._total_tokens}")

        # Підрахунок N-грамів (рівні 2 до n)
        for i in range(2, self.n + 1):
            self._ngram_counts[i] = Counter()
            for sent in processed_sentences:
                # Перебираємо вікна довжиною i
                for j in range(len(sent) - i + 1):
                    ngram = tuple(sent[j : j + i])
                    self._ngram_counts[i][ngram] += 1
            print(f"  Підраховано {len(self._ngram_counts[i])} унікальних {i}-грам.")


    def _map_token_to_vocab(self, token: str) -> str:
        return token if token in self.vocabulary else UNK_TOKEN

    def get_count(self, tokens: tuple) -> int:
        if not tokens: return self._total_tokens # count(()) - загальна кількість токенів

        n = len(tokens)
        # Мапінг вхідних токенів на словник моделі
        processed_tokens = tuple(self._map_token_to_vocab(token) for token in tokens)

        if n == 1: return self._ngram_counts.get(1, {}).get(processed_tokens[0], 0)
        elif n >= 2 and n <= self.n: return self._ngram_counts.get(n, {}).get(processed_tokens, 0)
        else: return 0 # Рівень вищий за n

    def prob(self, ngram: tuple) -> float:
        raise NotImplementedError("Метод 'prob' має бути реалізований у похідному класі.")

    def perplexity(self, sentences: list[list[str]]) -> float:
        N = 0       # Кількість токенів для оцінки
        log_prob_sum = 0.0 # Сума логарифмів ймовірностей

        for sent in sentences:
            processed_tokens = [self._map_token_to_vocab(token) for token in sent]
            sentence_with_specials = [START_TOKEN] * (self.n - 1) + processed_tokens + [END_TOKEN]
            num_ngrams_in_sent = len(sentence_with_specials) - (self.n - 1)
            if num_ngrams_in_sent <= 0: continue # Пропускаємо занадто короткі речення

            N += num_ngrams_in_sent

            # Обчислення логарифмів ймовірностей для кожної N-грами
            for i in range(len(sentence_with_specials) - self.n + 1):
                ngram = tuple(sentence_with_specials[i : i + self.n])
                p = self.prob(ngram) # Використовуємо prob() конкретної моделі
                if p == 0:
                     return float('inf')

                log_prob_sum += math.log(p)

        if N == 0:
            print("Попередження: Немає N-грамів для оцінки в тестовому наборі.")
            return float('inf') # Неможливо обчислити Perplexity

        # Perplexity = exp(- (1/N) * log_prob_sum)
        perplexity_value = math.exp(-log_prob_sum / N)
        print(f"Perplexity обчислено для {N} N-грам.")
        return perplexity_value

    def __repr__(self) -> str:
         return f"{self.__class__.__name__}(n={self.n}, unk_threshold={self.unk_threshold})"


#Модель зі згладжуванням Лапласа (add-k)
class LaplaceSmoothedNGramModel(BaseNGramModel):
    """
    N-грамна модель зі згладжуванням Лапласа (add-k).
    """
    def __init__(self, n: int, smoothing_k: float = 1.0, unk_threshold: int = 1):
        super().__init__(n, unk_threshold)
        if smoothing_k < 0: raise ValueError("smoothing_k >= 0")
        self.smoothing_k = smoothing_k

    def prob(self, ngram: tuple) -> float:
        """
        Обчислює P(w | context) = (count(ngram) + k) / (count(context) + k * |V|)
        """
        if len(ngram) != self.n:
            return 0.0

        ngram_count = self.get_count(ngram) # Вже оброблено UNK всередині get_count

        # Контекст - перші N-1 токенів N-грами
        context = ngram[:-1]
        context_count = self.get_count(context) # Вже оброблено UNK всередині get_count

        # Розмір словника для згладжування (включаючи UNK, START, END)
        vocab_size = len(self.vocabulary)

        # Захист від ділення на нуль, хоча з k > 0 та vocab_size > 0 це можливо тільки якщо context_count + k*|V| = 0
        # що малоймовірно, якщо словник не порожній.
        denominator = context_count + self.smoothing_k * vocab_size
        if denominator == 0:
            return 0.0 if vocab_size > 0 else 0.0

        return (ngram_count + self.smoothing_k) / denominator

    def __repr__(self) -> str:
         return f"LaplaceSmoothedNGramModel(n={self.n}, k={self.smoothing_k}, unk_threshold={self.unk_threshold}, vocab_size={len(self.vocabulary)})"


# Модель з бекофом (Stupid Backoff)
class StupidBackoffModel(BaseNGramModel):
    """
    N-грамна модель з тупим бекофом.
    P_bo(w|context) = count(context, w) / count(context) if count(context, w) > 0
                    = alpha * P_bo(w|context[1:]) otherwise
    """
    def __init__(self, n: int, alpha: float = 0.4, unk_threshold: int = 1):
        """
        alpha: коефіцієнт дисконтування при бекофі.
        """
        super().__init__(n, unk_threshold)
        if not 0 <= alpha <= 1: raise ValueError("alpha має бути в [0, 1]")
        self.alpha = alpha

    def prob(self, ngram: tuple) -> float:
        current_n = len(ngram)

        # Базовий випадок: Unigram (рівень 1)
        if current_n == 1:
            word = ngram[0]
            word_count = self.get_count((word,)) # count(word)
            total_count = self.get_count(())   # Загальна кількість токенів
            return word_count / total_count if total_count > 0 else 0.0

        ngram_count = self.get_count(ngram)
        context = ngram[:-1]
        context_count = self.get_count(context)

        # Якщо N-грама знайдена, використовуємо MLE (без згладжування на цьому рівні)
        if ngram_count > 0:
             # Перевірка context_count > 0 для безпеки, хоча якщо ngram_count>0, context_count має бути >= ngram_count
             return ngram_count / context_count if context_count > 0 else 0.0 # Якщо context_count=0, щось не так з даними/підрахунком
        # Якщо N-грама не знайдена, бекоф до нижчого рівня
        else:
            # Бекоф до ngram без першого токена
            return self.alpha * self.prob(ngram[1:]) # Рекурсивний виклик

    def __repr__(self) -> str:
         return f"StupidBackoffModel(n={self.n}, alpha={self.alpha}, unk_threshold={self.unk_threshold}, vocab_size={len(self.vocabulary)})"


#Модель з лінійною інтерполяцією 
class InterpolatedNGramModel(BaseNGramModel):
    """
    N-грамна модель з лінійною інтерполяцією.
    P_interp(w|context) = sum(lambda_k * P_k(w|last k-1 context tokens)) for k=1..n
    """
    def __init__(self, n: int, lambdas: list[float], unk_threshold: int = 1):
        """
        lambdas: список коефіцієнтів інтерполяції для рівнів 1 до n.
                 Сума lambdas має бути близька до 1.
        """
        if len(lambdas) != n: raise ValueError(f"Кількість коефіцієнтів lambda має дорівнювати n ({n}), отримано {len(lambdas)}.")
        if not math.isclose(sum(lambdas), 1.0): print(f"Попередження: Сума коефіцієнтів lambda ({sum(lambdas):.4f}) не дорівнює 1.")
        if any(lam < 0 for lam in lambdas): raise ValueError("Коефіцієнти lambda мають бути невід'ємними.")

        super().__init__(n, unk_threshold)
        self.lambdas = lambdas
        self._base_model_ref = None # Буде посилатись на self після fit

    def prob(self, ngram: tuple) -> float:
        if len(ngram) != self.n:
            print(f"Попередження: ngram довжини {len(ngram)} подано в модель рівня {self.n}. Очікується довжина {self.n}.")
            return 0.0 

        total_prob = 0.0
        # Ітеруємо по рівнях N-грам від 1 до n
        for k in range(1, self.n + 1):
            # Беремо останні k токенів з ngram
            sub_ngram = ngram[self.n - k :]
            # Контекст для k-грами - перші k-1 токенів sub_ngram
            sub_context = sub_ngram[:-1]
            sub_word = sub_ngram[-1] # Слово, яке прогнозуємо

            # Отримуємо count(sub_ngram) та count(sub_context) використовуючи загальні лічильники
            sub_ngram_count = self.get_count(sub_ngram)
            sub_context_count = self.get_count(sub_context)

            # Обчислюємо MLE для k-грами
            mle_k = sub_ngram_count / sub_context_count if sub_context_count > 0 else 0.0
            # Якщо k=1 (уніграма), sub_context порожній, count(sub_context) = total_tokens
            if k == 1:
                 mle_k = sub_ngram_count / self.get_count(()) if self.get_count(()) > 0 else 0.0
            # Індекси lambdas: lambdas[0] для 1-грами, lambdas[1] для 2-грами, ..., lambdas[n-1] для n-грами
            total_prob += self.lambdas[k - 1] * mle_k

        return total_prob

    def __repr__(self) -> str:
         return f"InterpolatedNGramModel(n={self.n}, lambdas={self.lambdas}, unk_threshold={self.unk_threshold}, vocab_size={len(self.vocabulary)})"


#Завдання 3: Генерація речень
def generate_sentence(model: BaseNGramModel, prompt: str = "", max_length: int = 20) -> str:
    print(f" Генерація: '{prompt}' (макс. {max_length} токенів)")

    # Токенізація та обробка промпта (UNK)
    prompt_tokens = [model._map_token_to_vocab(token.lower()) for token in re.findall(r'\w+|[.,!?;:\'"()-]', prompt)]

    # Початкова послідовність для контексту (включаючи START токени)
    num_start = max(0, model.n - 1 - len(prompt_tokens))
    current_sequence = [START_TOKEN] * num_start + prompt_tokens

    generated_tokens = list(prompt_tokens) # Зберігаємо тільки згенеровані (та промпт)

    # Цикл генерації
    for _ in range(max_length):
        # Поточний контекст (останні N-1 токенів)
        context = tuple(current_sequence[-(model.n - 1):])

        # Обчислення ймовірностей для наступного слова
        candidates_probs = {}
        for word in model.vocabulary:
            # Не генеруємо START
            if word == START_TOKEN: continue
            # Ймовірність N-грами (контекст + слово)
            p = model.prob(context + (word,))
            candidates_probs[word] = p

        # Вибірка наступного слова за ймовірністю
        words = list(candidates_probs.keys())
        probs = list(candidates_probs.values())

        # Нормалізація для random.choices
        total_prob = sum(probs)
        if total_prob == 0:
            break # Неможливо обрати наступне слово

        probs = [p / total_prob for p in probs]

        try:
            next_token = random.choices(words, weights=probs, k=1)[0]
        except ValueError as e:
             print(f"Помилка вибірки: {e}. Проблеми з ймовірностями? Зупинка.")
             break

        # Додаємо токен до послідовності та згенерованих
        current_sequence.append(next_token)
        generated_tokens.append(next_token)

        # Зупинка при END_TOKEN
        if next_token == END_TOKEN:
            break

    display_tokens = [token for token in generated_tokens if token not in {START_TOKEN, END_TOKEN, UNK_TOKEN}]
    generated_text = " ".join(display_tokens)
    # Проста обробка пунктуації (приєднуємо до попереднього слова)
    generated_text = re.sub(r'\s+([.,!?;:\'"()-])', r'\1', generated_text)

    return generated_text.capitalize()


if __name__ == '__main__':
    sents = load_nltk_sentences('brown')
    # Обмежуємо для швидкості виконання прикладу
    if len(sents) > 50000: sents = sents[:50000]

    # Розподіл на тренувальний та тестовий набори
    random.seed(42) # Для відтворюваності
    random.shuffle(sents)
    split_idx = int(0.8 * len(sents))
    train_sents, test_sents = sents[:split_idx], sents[split_idx:]
    print(f"Набори даних: Тренувальний={len(train_sents)} речень, Тестовий={len(test_sents)} речень.")

    #Завдання 0 & Частина 2: Побудова базової N-грам моделі зі згладжуванням 
    print("Завдання 0 & 2 (Laplace Smoothed) ")
    # N=3 модель зі згладжуванням Лапласа (k=1) та UNK порогом 1
    # UNK слова будуть замінені на <unk> під час fit()
    laplace_model = LaplaceSmoothedNGramModel(n=3, smoothing_k=1.0, unk_threshold=1)
    laplace_model.fit(train_sents)

    # Оцінка на тестовому наборі
    laplace_perplexity = laplace_model.perplexity(test_sents)
    print(f"Laplace Smoothed (k=1) Trigram Perplexity на тесті: {laplace_perplexity:.2f}")


    # Завдання 1: Порівняння бі- та триграмних моделей (за Perplexity на тесті) 
    print("Завдання 1: Порівняння моделей (на тесті)")
    # Тренуємо окремі бі- та триграмні моделі (зі згладжуванням для можливості оцінки)
    # Використовуємо той самий UNK поріг для чесного порівняння
    bigram_model_comp = LaplaceSmoothedNGramModel(n=2, smoothing_k=1.0, unk_threshold=1)
    bigram_model_comp.fit(train_sents)
    bigram_perplexity_test = bigram_model_comp.perplexity(test_sents)
    print(f"Bigram (N=2, k=1) Perplexity на тесті: {bigram_perplexity_test:.2f}")

    trigram_model_comp = LaplaceSmoothedNGramModel(n=3, smoothing_k=1.0, unk_threshold=1)
    trigram_model_comp.fit(train_sents) # Вже зроблено вище як laplace_model, але повторимо для ясності порівняння
    trigram_perplexity_test = trigram_model_comp.perplexity(test_sents)
    print(f"Trigram (N=3, k=1) Perplexity на тесті: {trigram_perplexity_test:.2f}")

    print("\nВисновок Завдання 1: Модель з меншою Perplexity краще передбачає наступне слово.")


    # Завдання 2: Інші методи згладжування/бекофу (Stupid Backoff, Interpolation)
    print("Завдання 2: Методи Interpolation та Stupid Backoff")

    # Модель Stupid Backoff (N=3)
    # Ця модель НЕ використовує add-k. Її prob() реалізує логіку бекофу.
    stupid_backoff_model = StupidBackoffModel(n=3, alpha=0.4, unk_threshold=1)
    stupid_backoff_model.fit(train_sents)
    stupid_backoff_perplexity = stupid_backoff_model.perplexity(test_sents)
    print(f"Stupid Backoff (N=3, alpha=0.4) Perplexity на тесті: {stupid_backoff_perplexity:.2f}")


    # Модель з лінійною інтерполяцією (N=3)
    interpolation_lambdas = [0.1, 0.3, 0.6] # lambda_1, lambda_2, lambda_3
    interpolated_model = InterpolatedNGramModel(n=3, lambdas=interpolation_lambdas, unk_threshold=1)
    interpolated_model.fit(train_sents) # fit() базового класу підраховує всі лічильники для всіх рівнів
    interpolated_perplexity = interpolated_model.perplexity(test_sents)
    print(f"Interpolated (N=3, lambdas={interpolation_lambdas}) Perplexity на тесті: {interpolated_perplexity:.2f}")

    print("\nВисновок Завдання 2: Методи Interpolation та Backoff краще обробляють рідкісні N-грами")


    # Завдання 3: Генерація речень 
    print("Завдання 3: Генерація речень за моделями")

    # Генерація за моделлю зі згладжуванням Лапласа (LaplaceSmoothedNGramModel)
    generated_laplace_1 = generate_sentence(laplace_model, prompt="the government")
    print(f"Промпт: 'the government' -> '{generated_laplace_1}'")

    generated_laplace_2 = generate_sentence(laplace_model, prompt="machine learning is", max_length=15)
    print(f"Промпт: 'machine learning is' -> '{generated_laplace_2}'")

    # Генерація за моделлю Stupid Backoff (StupidBackoffModel)
    print("\nГенерація за Stupid Backoff Model:")
    generated_backoff_1 = generate_sentence(stupid_backoff_model, prompt="the government")
    print(f"Промпт: 'the government' -> '{generated_backoff_1}'")

    generated_backoff_2 = generate_sentence(stupid_backoff_model, prompt="new possibilities in", max_length=15)
    print(f"Промпт: 'new possibilities in' -> '{generated_backoff_2}'")

    # Генерація за моделлю Interpolated (InterpolatedNGramModel)
    print("\nГенерація за Interpolated Model:")
    generated_interpolated_1 = generate_sentence(interpolated_model, prompt="and make intelligent")
    print(f"Промпт: 'and make intelligent' -> '{generated_interpolated_1}'")

    generated_interpolated_2 = generate_sentence(interpolated_model, prompt="this is a", max_length=10)
    print(f"Промпт: 'this is a' -> '{generated_interpolated_2}'")

    print(" Приклад використання UNK")
    print("\nГенерація з промптом, що може вести до UNK (використовуючи Backoff):")
    generated_unk = generate_sentence(stupid_backoff_model, prompt="the young", max_length=10)
    print(f"Промпт: 'the young' -> '{generated_unk}'") # 'young' може бути рідкісним


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


Завантаження корпусу 'brown'...
Завантажено 57340 речень.
Набори даних: Тренувальний=40000 речень, Тестовий=10000 речень.
Завдання 0 & 2 (Laplace Smoothed) 
Навчання моделі (N=3, UNK Threshold=1)...
  Розмір словника (включаючи спецтокени, UNK): 23238
  Загальна кількість токенів після обробки: 951437
  Підраховано 309737 унікальних 2-грам.
  Підраховано 627538 унікальних 3-грам.
Perplexity обчислено для 218483 N-грам.
Laplace Smoothed (k=1) Trigram Perplexity на тесті: 9457.72
Завдання 1: Порівняння моделей (на тесті)
Навчання моделі (N=2, UNK Threshold=1)...
  Розмір словника (включаючи спецтокени, UNK): 23238
  Загальна кількість токенів після обробки: 911437
  Підраховано 309736 унікальних 2-грам.
Perplexity обчислено для 218483 N-грам.
Bigram (N=2, k=1) Perplexity на тесті: 1708.18
Навчання моделі (N=3, UNK Threshold=1)...
  Розмір словника (включаючи спецтокени, UNK): 23238
  Загальна кількість токенів після обробки: 951437
  Підраховано 309737 унікальних 2-грам.
  Підраховано 62